# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [1]:
# all import statements needed for the project, for example:
import os
import re
import requests
import bs4
from bs4 import BeautifulSoup
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import sqlalchemy as db
import pyarrow.parquet as pq
from datetime import datetime
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text
import folium
from folium.plugins import HeatMap
from IPython.display import display
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from typing import List, Optional, Tuple, Dict, Any, Union

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### 1.1 Downloading data

#### 1.1.1 Define the function ***filter_links_by_date*** to scrap the data only from January 2020 to August 2024

In [ ]:
def filter_links_by_date(
    links: List[str], 
    start_year: int, 
    start_month: int, 
    end_year: int, 
    end_month: int
) -> List[str]:
    """
    Filters a list of links to include only those within the specified date range.

    Args:
        links (list): List of URLs to filter.
        start_year (int): Starting year of the range (inclusive).
        start_month (int): Starting month of the range (inclusive).
        end_year (int): Ending year of the range (inclusive).
        end_month (int): Ending month of the range (inclusive).

    Returns:
        list: Filtered list of URLs.
    """
    filtered_links = []
    for link in links:
        # Extract year and month using regex
        match = re.search(r"(\d{4})-(\d{2})", link)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            # Check if the year and month fall within the specified range
            if (start_year < year < end_year) or \
               (year == start_year and start_month <= month) or \
               (year == end_year and month <= end_month):
                filtered_links.append(link)
    return filtered_links

start_year, start_month = 2020, 1
end_year, end_month = 2024, 8

#### 1.1.2 Extract the file links for Yellow Taxi & Uber trip data from the TLC Trip Record Data web pages and then download the filtered dated files to save them to a local directory.

In [ ]:
# URL of the NYC Trip Data page
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Define function to download Parquet files
def download_parquet_files(links: List[str], save_dir: str):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for link in links:
        file_name = link.split("/")[-1]
        file_path = os.path.join(save_dir, file_name)
        print(f"Downloading {file_name}...")
        
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(link, headers=headers)
        
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"Saved to {file_path}")
        else:
            print(f"Failed to download {file_name}: HTTP {response.status_code}")

# Scrape the webpage
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all anchor tags with href attributes
all_links = [a["href"].strip() for a in soup.find_all("a", href=True)]

# Include only Yellow Taxi and HVFHV trip data using re module
yellow_taxi_pattern = r".*yellow_tripdata.*\.parquet"
hvfhv_pattern = r".*fhvhv_tripdata.*\.parquet"

yellow_taxi_links = [link for link in all_links if re.search(yellow_taxi_pattern, link)]
filtered_yellow_taxi_links = filter_links_by_date(yellow_taxi_links, start_year, start_month, end_year, end_month)
hvfhv_links = [link for link in all_links if re.search(hvfhv_pattern, link)]
filtered_hvfhv_links = filter_links_by_date(hvfhv_links, start_year, start_month, end_year, end_month)

# Make full URLs if relative paths are present
base_url = "https://www.nyc.gov"
yellow_taxi_links = [
    link if link.startswith("http") else base_url + link for link in filtered_yellow_taxi_links
]
hvfhv_links = [
    link if link.startswith("http") else base_url + link for link in hvfhv_links
]

# Debugging: Print the found links
print("Yellow Taxi Links:", filtered_yellow_taxi_links)
print("HVFHV Links:", filtered_hvfhv_links)

# Download Parquet files
print("Downloading Yellow Taxi Parquet files...")
download_parquet_files(filtered_yellow_taxi_links, save_dir = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project")

print("\nDownloading HVFHV Parquet files...")
download_parquet_files(filtered_hvfhv_links, save_dir = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project")

#### 1.1.3 Loading the Taxi Zones shapefile

In [ ]:
# Path to the Taxi Zones shapefile
TAXI_ZONES_SHAPEFILE = "taxi_zones.shp"

# Load the shapefile into a GeoDataFrame
taxi_zones_gdf = gpd.read_file(TAXI_ZONES_SHAPEFILE)
taxi_zones_gdf = taxi_zones_gdf.to_crs(epsg=4326)

# Inspect the GeoDataFrame
print(taxi_zones_gdf.head())

### 1.2 Related function

#### 1.2.1 Load Taxi Zones

In [ ]:
def load_taxi_zones(shapefile: str) -> Optional[gpd.GeoDataFrame]:
    """
    Load and process the taxi zones shapefile, extracting longitude and latitude from the geometry column.

    Parameters:
        shapefile_path (str): Path to the taxi zones shapefile.

    Returns:
        gpd.GeoDataFrame: Processed GeoDataFrame with longitude and latitude columns added.
    """
    try:
        # Load the Taxi Zones GeoDataFrame
        taxi_zones_gdf = gpd.read_file(shapefile)

        # Check CRS and reproject to WGS84 (EPSG:4326) if necessary
        if taxi_zones_gdf.crs != "EPSG:4326":
            taxi_zones_gdf = taxi_zones_gdf.to_crs(epsg=4326)

        # Extract longitude and latitude from the geometry column
        taxi_zones_gdf["longitude"] = taxi_zones_gdf.geometry.centroid.x
        taxi_zones_gdf["latitude"] = taxi_zones_gdf.geometry.centroid.y

        return taxi_zones_gdf

    except Exception as e:
        print(f"Error loading and processing shapefile: {e}")
        return None

In [ ]:
def lookup_coords_for_taxi_zone_id(
    zone_loc_id: int, 
    loaded_taxi_zones: gpd.GeoDataFrame
) -> Tuple[Optional[float], Optional[float]]:
    """
    Look up the coordinates (longitude, latitude) of a taxi zone given its Location ID.

    Parameters:
    - zone_loc_id (int): The Location ID of the taxi zone.
    - loaded_taxi_zones (gpd.GeoDataFrame): The GeoDataFrame containing the taxi zones.

    Returns:
    - tuple: A tuple (longitude, latitude) representing the coordinates of the taxi zone.
    """
    try:
        # Filter the GeoDataFrame for the given Location ID
        zone = loaded_taxi_zones[loaded_taxi_zones["LocationID"] == zone_loc_id]

        # Check if the zone is valid
        if zone.empty:
            raise ValueError(f"Location ID {zone_loc_id} not found in the Taxi Zones dataset.")

        # Use the centroid of the zone polygon for coordinates
        longitude = zone.geometry.centroid.x.values[0]
        latitude = zone.geometry.centroid.y.values[0]

        # Return as a tuple
        return longitude, latitude

    except Exception as e:
        print(f"Error looking up coordinates for Location ID {zone_loc_id}: {e}")
        return None, None  # Return None for both longitude and latitude if there's an error


#### 1.2.2 Calculate Sample Size

In [ ]:
def calculate_taxi_sample_size(population: Union[int, float]) -> int:
    '''
    Calculate the sample size needed for a given population

    Parameters:
    - population: total population size

    Returns:
    - the sample size needed
    '''
    # set confidence_level = 0.95
    margin_of_error = 0.05
    z_score = 1.96 # for 95% confidence level
    proportion = 0.5 # proportion of the population that has the attribute of interest (use 0.5 for max variance)

    # implement Cochran's formula
    sample_size = (z_score**2 * proportion * (1 - proportion)) / (margin_of_error**2)

    return math.ceil(sample_size)

In [ ]:
def calculate_fhvhv_sample_size(population: Union[int, float]) -> int:
    '''
    Calculate the sample size needed for a given population

    Parameters:
    - population: total population size

    Returns:
    - the sample size needed
    '''
    # set confidence_level = 0.99
    margin_of_error = 0.05
    z_score = 2.58 # for 95% confidence level
    proportion = 0.5 # proportion of the population that has the attribute of interest (use 0.5 for max variance)

    # implement Cochran's formula
    sample_size = (z_score**2 * proportion * (1 - proportion)) / (margin_of_error**2)

    return math.ceil(sample_size)

### 1.2.3 Common Functions

In [ ]:
def get_all_urls_from_tlc_page(taxi_page: str) -> List[str]:
    '''
    Get all the URLs from the TLC page
    
    Parameters:
    - taxi_page: the URL of the TLC page
    
    Returns:
    - a list of all URLs on the page
    '''
    
    response = requests.get(taxi_page)
    soup = BeautifulSoup(response.content, "html.parser")
    all_links = [a["href"] for a in soup.find_all("a", href=True)]
    
    return all_links

In [ ]:
def filter_parquet_urls(all_urls: List[str]) -> Dict[str, List[str]]:
    '''
    Filter the URLs to get only the Parquet files
    
    Parameters:
    - all_urls: a list of all URLs
    
    Returns:
    - a list of URLs that contain the word "parquet"
    '''
    
    # Define patterns for Yellow Taxi and HVFHV Parquet files
    yellow_taxi_pattern = r".*yellow_tripdata.*\.parquet"
    hvfhv_pattern = r".*fhvhv_tripdata.*\.parquet"

    # Filter URLs for each dataset
    yellow_taxi_urls = [url for url in all_urls if re.search(yellow_taxi_pattern, url)]
    hvfhv_urls = [url for url in all_urls if re.search(hvfhv_pattern, url)]

    return {
        "yellow_taxi": yellow_taxi_urls,
        "hvfhv": hvfhv_urls,
    }

### 1.3 Process Taxi Data

#### 1.3.1 Generate random samples of taxi

In [ ]:
# Define the directory containing the taxi monthly datasets
directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
output_directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over each file in the directory
for file_name in os.listdir(directory):
    if "yellow" in file_name and file_name.endswith(".parquet"):
        file_path = os.path.join(directory, file_name)

        # Read the dataset
        data = pd.read_parquet(file_path)
        population_size = len(data)  # Get total rows in the dataset

        # Calculate sample size dynamically
        sample_size = calculate_taxi_sample_size(population_size)

        # Create a random sample
        sampled_data = data.sample(n=sample_size)

        # Save the sampled data to a new file
        output_path = os.path.join(output_directory, f"sampled_{file_name}")
        sampled_data.to_parquet(output_path)

        # Print information for debugging
        print(f"Processed {file_name}: Population = {population_size}, Sample = {sample_size}")


#### 1.3.2 Cleaning and processing taxi sample data

In [ ]:
# NYC bounding box coordinates (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LAT_MIN, LON_MIN = NEW_YORK_BOX_COORDS[0]
LAT_MAX, LON_MAX = NEW_YORK_BOX_COORDS[1]
CRS = 4326

def get_and_clean_taxi_data(
    file_path: str, 
    save_dir: str, 
    taxi_zones_gdf: gpd.GeoDataFrame
) -> Optional[pd.DataFrame]:
    """
    Load and clean a single month's taxi data from a local Parquet file.

    Parameters:
        file_path (str): Path to the Parquet file.
        save_dir (str): Directory where cleaned files will be saved.
        taxi_zones_gdf (gpd.GeoDataFrame): GeoDataFrame of taxi zones.

    Returns:
        pd.DataFrame: Cleaned DataFrame for the given month.
    """
    file_name = os.path.basename(file_path)

    try:
        # Step 1: Load the Parquet file
        print(f"Loading file: {file_name}...")
        df = pd.read_parquet(file_path)

        # Step 2: Normalize column names
        df.columns = df.columns.str.strip().str.lower()

        # Step 3: Look up and add coordinates for pulocationid and dolocationid
        if "pulocationid" in df.columns and "dolocationid" in df.columns:
            valid_location_ids = taxi_zones_gdf["LocationID"]
            initial_rows = len(df)
            df = df[df["pulocationid"].isin(valid_location_ids) & df["dolocationid"].isin(valid_location_ids)]
            dropped_rows = initial_rows - len(df)
            print(f"Dropped {dropped_rows} rows with invalid Location IDs.")
        else:
            print(f"Columns 'pulocationid' or 'dolocationid' not found in {file_name}. Skipping location ID filtering.")
            return None

        # Step 5: Drop rows with the same pickup and dropoff location
        initial_rows = len(df)
        df = df[df["pulocationid"] != df["dolocationid"]]
        filtered_rows = initial_rows - len(df)
        print(f"Filtered out {filtered_rows} rows with the same pickup and dropoff location.")

        # Step 6: Look up and add coordinates for pulocationid and dolocationid
        print("Looking up coordinates for location IDs...")
        df["pickup_coords"] = df["pulocationid"].apply(
            lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, taxi_zones_gdf)
        )
        df["dropoff_coords"] = df["dolocationid"].apply(
            lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, taxi_zones_gdf)
        )

        # Split coordinates into latitude and longitude columns
        df[["pickup_longitude", "pickup_latitude"]] = pd.DataFrame(
            df["pickup_coords"].tolist(), index=df.index
        )
        df[["dropoff_longitude", "dropoff_latitude"]] = pd.DataFrame(
            df["dropoff_coords"].tolist(), index=df.index
        )

        # Drop temporary coordinate columns
        df.drop(["pickup_coords", "dropoff_coords"], axis=1, inplace=True)


        # Step 7: Remove unnecessary columns
        columns_to_keep = [
            'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
            'trip_distance', 'pulocationid', 'dolocationid', 'fare_amount', 'extra',
            'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 
            'congestion_surcharge', 'airport_fee', 'pickup_latitude', 'pickup_longitude',
            'dropoff_latitude', 'dropoff_longitude'
        ]
        df = df[[col for col in columns_to_keep if col in df.columns]]

        # Step 8: Normalize datetime columns
        for time_col in ['tpep_pickup_datetime', 'tpep_dropoff_datetime']:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
                
        if "airport_fee" in df.columns:
            df["airport_fee"] = df["airport_fee"].fillna(0) 
            
        # Step 9: Filter trips within NYC bounding box
        if {"pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"}.issubset(df.columns):
            df = df[
                (df['pickup_latitude'] >= LAT_MIN) & (df['pickup_latitude'] <= LAT_MAX) &
                (df['pickup_longitude'] >= LON_MIN) & (df['pickup_longitude'] <= LON_MAX) &
                (df['dropoff_latitude'] >= LAT_MIN) & (df['dropoff_latitude'] <= LAT_MAX) &
                (df['dropoff_longitude'] >= LON_MIN) & (df['dropoff_longitude'] <= LON_MAX)
            ]

        # Step 10: Remove rides with the same pickup and dropoff locations or zero distance
        if "pulocationid" in df.columns and "dolocationid" in df.columns:
            df = df[df["pulocationid"] != df["dolocationid"]]
        if "trip_distance" in df.columns:
            df = df[df["trip_distance"] > 0]

        # Step 11: Rename columns for clarity
        df.rename(columns={
            "tpep_pickup_datetime": "pickup_time",
            "tpep_dropoff_datetime": "dropoff_time",
            "pulocationid": "pickup_location_id",
            "dolocationid": "dropoff_location_id",
            "extra": "miscellaneous_extra_charges"
        }, inplace=True)

        # Save the cleaned data
        output_path = os.path.join(save_dir, f"cleaned_{file_name}")
        df.to_parquet(output_path)
        print(f"Cleaned {len(df)} rows and saved to {output_path}.")
        return df

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
        return None


# Main Script
directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
output_directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
os.makedirs(output_directory, exist_ok=True)

# Load the Taxi Zones GeoDataFrame
TAXI_ZONES_SHAPEFILE = os.path.join(directory, "taxi_zones.shp")
taxi_zones_gdf = load_taxi_zones(TAXI_ZONES_SHAPEFILE)

# Iterate over each Parquet file in the directory
for file_name in os.listdir(directory):
    # Process only files that contain "yellow" and end with ".parquet"
    if "sampled_yellow" in file_name.lower() and file_name.endswith(".parquet"):
        file_path = os.path.join(directory, file_name)
        get_and_clean_taxi_data(file_path, output_directory, taxi_zones_gdf)

#### 1.3.3 Merge the cleaned .parquet files into a Pandas DataFrame

In [ ]:
def get_taxi_data(cleaned_files_dir: str) -> pd.DataFrame:
    """
    Load and combine cleaned taxi data from local cleaned Parquet files.

    Parameters:
        cleaned_files_dir (str): Directory where cleaned Parquet files are stored.

    Returns:
        pd.DataFrame: Combined DataFrame of all months.
    """
    all_taxi_dataframes = []

    # Iterate through all cleaned Parquet files in the directory
    for file_name in os.listdir(cleaned_files_dir):
        if file_name.startswith("cleaned_") and file_name.endswith(".parquet"):
            file_path = os.path.join(cleaned_files_dir, file_name)
            print(f"Loading cleaned file: {file_path}...")
            
            # Load the cleaned Parquet file into a DataFrame
            df = pd.read_parquet(file_path)
            all_taxi_dataframes.append(df)

    # Combine all cleaned data into a single DataFrame
    if all_taxi_dataframes:
        taxi_data = pd.concat(all_taxi_dataframes, ignore_index=True)
        print(f"Combined {len(taxi_data)} rows from all cleaned files.")
    else:
        taxi_data = pd.DataFrame()
        print("No cleaned files found to process.")

    return taxi_data


In [ ]:
def get_all_taxi_data(cleaned_files_dir: str) -> pd.DataFrame:
    """
    Combine all cleaned Yellow Taxi and HVFHV data from cleaned Parquet files.

    Parameters:
        cleaned_files_dir (str): Directory where cleaned Parquet files are stored.

    Returns:
        pd.DataFrame: Combined DataFrame of all cleaned taxi data.
    """
    # Process all cleaned Parquet files in the directory
    print("Combining all cleaned taxi data...")
    all_data = get_taxi_data(cleaned_files_dir=cleaned_files_dir)

    print(f"Final combined data contains {len(all_data)} rows.")
    return all_data


In [ ]:
CLEANED_FILES_DIR = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"

# Load and combine all cleaned data into a single DataFrame
taxi_data = get_all_taxi_data(cleaned_files_dir=CLEANED_FILES_DIR)

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### 1.4 Processing Uber Data

#### 1.4.1 Generate random samples of uber

In [ ]:
# Define the directory containing the taxi monthly datasets
directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
output_directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over each file in the directory
for file_name in os.listdir(directory):
    if "fhvhv" in file_name and file_name.endswith(".parquet"):
        file_path = os.path.join(directory, file_name)

        # Read the dataset
        data = pd.read_parquet(file_path)
        population_size = len(data)  # Get total rows in the dataset

        # Calculate sample size dynamically
        sample_size = calculate_fhvhv_sample_size(population_size)

        # Create a random sample
        sampled_data = data.sample(n=sample_size)

        # Save the sampled data to a new file
        output_path = os.path.join(output_directory, f"sampled_{file_name}")
        sampled_data.to_parquet(output_path)

        # Print information for debugging
        print(f"Processed {file_name}: Population = {population_size}, Sample = {sample_size}")


#### 1.4.2 Cleaning and processing uber sample data

In [ ]:
def get_and_clean_uber_data(
    file_path: str, 
    save_dir: str, 
    taxi_zones_gdf: gpd.GeoDataFrame
) -> Optional[pd.DataFrame]:
    """
    Load and clean a single month's Uber data from a local Parquet file.

    Parameters:
        file_path (str): Path to the Parquet file.
        save_dir (str): Directory where cleaned files will be saved.
        taxi_zones_gdf (gpd.GeoDataFrame): GeoDataFrame of taxi zones.

    Returns:
        pd.DataFrame: Cleaned DataFrame for the given month.
    """
    file_name = os.path.basename(file_path)

    try:
        # Step 1: Load the Parquet file
        print(f"Loading file: {file_name}...")
        df = pd.read_parquet(file_path)

        # Step 2: Normalize column names
        df.columns = df.columns.str.strip().str.lower()

        # Step 3: Filter out non-Uber rides
        if "hvfhs_license_num" in df.columns:
            df = df[df["hvfhs_license_num"].str.contains("HV0003", na=False)]
            print(f"Filtered to Uber rides only: {len(df)} rows remaining.")
        else:
            print(f"'hvfhs_license_num' column not found in {file_name}. Skipping Uber filtering.")
            return None

        # Step 4: Drop rows with invalid Location IDs
        if "pulocationid" in df.columns and "dolocationid" in df.columns:
            valid_location_ids = set(taxi_zones_gdf["LocationID"])
            initial_rows = len(df)
            df = df[df["pulocationid"].isin(valid_location_ids) & df["dolocationid"].isin(valid_location_ids)]
            dropped_rows = initial_rows - len(df)
            print(f"Dropped {dropped_rows} rows with invalid Location IDs.")
        else:
            print(f"Columns 'pulocationid' or 'dolocationid' not found in {file_name}. Skipping location ID filtering.")
            return None

        # Step 5: Drop rows with the same pickup and dropoff location
        initial_rows = len(df)
        df = df[df["pulocationid"] != df["dolocationid"]]
        filtered_rows = initial_rows - len(df)
        print(f"Filtered out {filtered_rows} rows with the same pickup and dropoff location.")

        # Step 6: Look up and add coordinates for pulocationid and dolocationid
        print("Looking up coordinates for location IDs...")
        df["pickup_coords"] = df["pulocationid"].apply(
            lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, taxi_zones_gdf)
        )
        df["dropoff_coords"] = df["dolocationid"].apply(
            lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, taxi_zones_gdf)
        )

        # Split coordinates into latitude and longitude columns
        df[["pickup_longitude", "pickup_latitude"]] = pd.DataFrame(
            df["pickup_coords"].tolist(), index=df.index
        )
        df[["dropoff_longitude", "dropoff_latitude"]] = pd.DataFrame(
            df["dropoff_coords"].tolist(), index=df.index
        )

        # Drop temporary coordinate columns
        df.drop(["pickup_coords", "dropoff_coords"], axis=1, inplace=True)

        # Step 7: Remove unnecessary columns
        columns_to_keep = [
            'pickup_datetime', 'dropoff_datetime', 'pulocationid', 'dolocationid',
            'trip_miles', 'base_passenger_fare', 'tolls', 'sales_tax', 
            'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
            'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'
        ]
        df = df[[col for col in columns_to_keep if col in df.columns]]

        # Step 8: Normalize datetime columns
        for time_col in ['pickup_datetime', 'dropoff_datetime']:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
        if 'trip_time' in df.columns:
            df['trip_time'] = pd.to_timedelta(df['trip_time'], errors='coerce')

        if "airport_fee" in df.columns:
            df["airport_fee"] = df["airport_fee"].fillna(0) 

        # Step 9: Filter trips within NYC bounding box
        if {"pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"}.issubset(df.columns):
            df = df[
                (df['pickup_latitude'] >= LAT_MIN) & (df['pickup_latitude'] <= LAT_MAX) &
                (df['pickup_longitude'] >= LON_MIN) & (df['pickup_longitude'] <= LON_MAX) &
                (df['dropoff_latitude'] >= LAT_MIN) & (df['dropoff_latitude'] <= LAT_MAX) &
                (df['dropoff_longitude'] >= LON_MIN) & (df['dropoff_longitude'] <= LON_MAX)
            ]

        # Step 10: Rename columns for clarity
        df.rename(columns={
            "pickup_datetime": "pickup_time",
            "dropoff_datetime": "dropoff_time",
            "pulocationid": "pickup_location_id",
            "dolocationid": "dropoff_location_id",
            "trip_miles": "trip_distance",
        }, inplace=True)

        # Save the cleaned data
        output_path = os.path.join(save_dir, f"cleaned_{file_name}")
        df.to_parquet(output_path)
        print(f"Cleaned {len(df)} rows and saved to {output_path}.")
        return df

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
        return None

# Main Script
directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
output_directory = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
os.makedirs(output_directory, exist_ok=True)

# Load the Taxi Zones GeoDataFrame
TAXI_ZONES_SHAPEFILE = os.path.join(directory, "taxi_zones.shp")
taxi_zones_gdf = load_taxi_zones(TAXI_ZONES_SHAPEFILE)

# Iterate over each Parquet file in the directory
for file_name in os.listdir(directory):
    # Process only files that contain "yellow" and end with ".parquet"
    if "sampled_fhvhv" in file_name.lower() and file_name.endswith(".parquet"):
        file_path = os.path.join(directory, file_name)
        get_and_clean_uber_data(file_path, output_directory, taxi_zones_gdf)

#### 1.4.3 Merge the cleaned .parquet files into a Pandas DataFrame

In [ ]:
def get_uber_data(
    cleaned_files_dir: str, 
    prefix: str = "cleaned_sampled_fhvhv", 
    file_extension: str = ".parquet"
) -> pd.DataFrame:
    """
    Load and combine cleaned Uber data from local cleaned Parquet files.

    Parameters:
        cleaned_files_dir (str): Directory where cleaned Parquet files are stored.
        prefix (str): Prefix to identify Uber files (default is "cleaned_uber_").
        file_extension (str): File extension to identify Parquet files (default is ".parquet").

    Returns:
        pd.DataFrame: Combined DataFrame of all months.
    """
    all_uber_dataframes = []

    # Verify that the directory exists
    if not os.path.exists(cleaned_files_dir):
        print(f"Directory does not exist: {cleaned_files_dir}")
        return pd.DataFrame()

    # Iterate through all files in the directory
    for file_name in os.listdir(cleaned_files_dir):
        # Filter files by prefix and file extension
        if file_name.startswith(prefix) and file_name.endswith(file_extension):
            file_path = os.path.join(cleaned_files_dir, file_name)
            print(f"Loading cleaned Uber file: {file_path}...")
            
            # Load the cleaned Parquet file into a DataFrame
            try:
                df = pd.read_parquet(file_path)
                all_uber_dataframes.append(df)
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")

    # Combine all cleaned data into a single DataFrame
    if all_uber_dataframes:
        uber_data = pd.concat(all_uber_dataframes, ignore_index=True)
        print(f"Combined {len(uber_data)} rows from all cleaned Uber files.")
    else:
        uber_data = pd.DataFrame()
        print("No cleaned Uber files found to process.")

    return uber_data


In [ ]:
def get_all_uber_data(
    cleaned_files_dir: str, 
    prefix: str = "cleaned_sampled_fhvhv", 
    file_extension: str = ".parquet"
) -> pd.DataFrame:
    """
    Combine all cleaned Uber data from cleaned Parquet files.

    Parameters:
        cleaned_files_dir (str): Directory where cleaned Parquet files are stored.
        prefix (str): Prefix to identify Uber files (default is "cleaned_uber_").
        file_extension (str): File extension to identify Parquet files (default is ".parquet").

    Returns:
        pd.DataFrame: Combined DataFrame of all cleaned Uber data.
    """
    # Check if the directory exists
    if not os.path.exists(cleaned_files_dir):
        print(f"Directory does not exist: {cleaned_files_dir}")
        return pd.DataFrame()

    # Combine all cleaned Uber data
    print(f"Combining all cleaned Uber data from directory: {cleaned_files_dir}...")
    all_uber_data = get_uber_data(cleaned_files_dir=cleaned_files_dir, prefix=prefix, file_extension=file_extension)

    # Check if the resulting DataFrame is empty
    if all_uber_data.empty:
        print("No Uber data found or combined. Returning an empty DataFrame.")
    else:
        print(f"Final combined Uber data contains {len(all_uber_data)} rows.")
    
    return all_uber_data


In [ ]:
# Main Script for Uber Data
CLEANED_UBER_FILES_DIR = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"

# Load and combine all cleaned Uber data into a single DataFrame
uber_data = get_all_uber_data(cleaned_files_dir=CLEANED_UBER_FILES_DIR)

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### 1.5 Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory: str) -> List[str]:
    """
    Returns a list of all CSV files in the given directory that are related to weather data.
    Weather-related files are identified by the presence of the keyword 'weather' in the filename.

    Args:
        directory (str): Path to the directory to search for files.

    Returns:
        list: A list of weather-related CSV filenames.
    """
    if not os.path.isdir(directory):
        raise ValueError(f"Invalid directory: {directory}")
    
    weather_csvs = [
        file for file in os.listdir(directory)
        if file.endswith('.csv') and 'weather' in file.lower()
    ]
    
    return weather_csvs

#### 1.5.1 Cleaning and processing weather data

##### Hourly

In [ ]:
def clean_month_weather_data_hourly(csv_file: str) -> pd.DataFrame:
    """
    Cleans the weather dataset by:
    - Extracting specific columns.
    - Normalizing column names by adding underscores between words.
    - Converting weather codes to descriptive terms.
    - Handling multiple data points within the same hour by selecting the ideal one.
    - Filling missing hourly data by interpolating between previous and next data points.

    Parameters:
        csv_file (str): Path to the CSV file containing the weather data.

    Returns:
        pd.DataFrame: A cleaned DataFrame with selected and transformed columns.
    """
    # AU code mapping for HourlyPresentWeatherType
    au_code_mapping = {
        "DZ": "Drizzle",
        "RA": "Rain",
        "SN": "Snow",
        "SG": "Snow Grains",
        "IC": "Ice Crystals",
        "PL": "Ice Pellets",
        "GR": "Hail",
        "GS": "Small Hail",
        "UP": "Unknown Precipitation",
        "BR": "Mist",
        "FG": "Fog",
        "FU": "Smoke",
        "VA": "Volcanic Ash",
        "DU": "Dust",
        "SA": "Sand",
        "HZ": "Haze",
        "PY": "Spray",
        "PO": "Sand Whirls",
        "SQ": "Squalls",
        "FC": "Funnel Cloud",
        "SS": "Sandstorm",
        "DS": "Duststorm"
    }

    # Sky condition mapping for HourlySkyConditions
    sky_condition_mapping = {
        "CLR": "Clear",
        "FEW": "Few Clouds",
        "SCT": "Scattered Clouds",
        "BKN": "Broken Clouds",
        "OVC": "Overcast",
        "VV": "Obscured Sky"
    }

    # Function to interpret HourlyPresentWeatherType using AU codes
    def interpret_weather_type(weather_string):
        if pd.isnull(weather_string):
            return "Unknown"
        matches = re.findall(r"([A-Z]{2}):\d+", weather_string)
        descriptions = [au_code_mapping.get(code, "Unknown") for code in matches]
        return ", ".join(set(descriptions)) if descriptions else "Unknown"

    # Function to interpret HourlySkyConditions
    def interpret_sky_conditions(sky_string):
        if pd.isnull(sky_string):
            return "Unknown"
        pattern = r"(\w{3}):(\d{2})(?:\s(\d+))?"
        matches = re.findall(pattern, sky_string)
        interpreted_conditions = []
        for condition, octa, elevation in matches:
            description = sky_condition_mapping.get(condition, "Unknown")
            detail = f"{description}, Octas: {int(octa)}"
            if elevation:
                detail += f", Elevation: {int(elevation)} feet"
            interpreted_conditions.append(detail)
        return "; ".join(interpreted_conditions) if interpreted_conditions else "Unknown"

    # Function to select the ideal row for each hour
    def select_ideal_row(group):
        if group.empty:
            return None
        if len(group) == 1:
            return group.iloc[0]
        middle_of_hour = group.index[0].replace(minute=30, second=0, microsecond=0)
        time_diffs = abs((group.index - middle_of_hour).total_seconds())
        return group.iloc[time_diffs.argmin()]

    # Load the dataset
    df = pd.read_csv(csv_file, low_memory=False)

    # Normalize column names (add underscores between words and make lowercase)
    df.columns = df.columns.str.replace(r"([a-z])([A-Z])", r"\1_\2", regex=True).str.lower().str.replace(" ", "_")

    # Rename columns for consistency
    column_renaming = {
        "date": "hourly_time",
        "hourly_dry_bulb_temperature": "hourly_temperature"
    }
    df.rename(columns=column_renaming, inplace=True)

    # Select only the required columns
    columns_to_extract = [
        "hourly_time", "hourly_temperature", "hourly_present_weather_type", 
        "hourly_sky_conditions", "hourly_visibility", "hourly_precipitation", "hourly_wind_speed"
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]

    # Convert hourly_time to datetime
    df["hourly_time"] = pd.to_datetime(df["hourly_time"], errors="coerce")
    df = df.dropna(subset=["hourly_time"])  # Drop rows with invalid dates
    df = df.sort_values("hourly_time")  # Sort by hourly_time
    df.set_index("hourly_time", inplace=True)  # Set hourly_time as the index

    # Group by hourly and select the ideal row
    df_hourly = df.groupby(pd.Grouper(freq='H')).apply(select_ideal_row)
    df_hourly = df_hourly.dropna()  # Drop any None rows from empty groups

    # Reindex to include all hourly intervals and interpolate missing data
    all_hours = pd.date_range(start=df_hourly.index.min(), end=df_hourly.index.max(), freq='H')
    df_hourly = df_hourly.reindex(all_hours)
    df_hourly.index.name = "hourly_time"

    # Replace NaN in hourly_precipitation with 0, and handle 'T' values
    if "hourly_precipitation" in df_hourly.columns:
    # Replace known non-numeric values
        df_hourly["hourly_precipitation"] = df_hourly["hourly_precipitation"].replace("T", "0.01")
        df_hourly["hourly_precipitation"] = df_hourly["hourly_precipitation"].replace("", "0")
    # Remove any leftover non-numeric characters (e.g., "s")
        df_hourly["hourly_precipitation"] = df_hourly["hourly_precipitation"].str.replace(r"[^\d.]", "", regex=True)
    # Convert to float, coercing any remaining errors to NaN
        df_hourly["hourly_precipitation"] = pd.to_numeric(df_hourly["hourly_precipitation"], errors="coerce").fillna(0)


    # Apply forward-fill for hourly_wind_speed -- assume wind doesn't change much within an hour
    if "hourly_wind_speed" in df_hourly.columns:
        df_hourly["hourly_wind_speed"] = df_hourly["hourly_wind_speed"].fillna(method='ffill')

    # Convert numerical columns to numeric, coercing errors to NaN
    numerical_cols = ["hourly_temperature", "hourly_visibility"]
    for col in numerical_cols:
        df_hourly[col] = pd.to_numeric(df_hourly[col], errors="coerce")

    # Interpolate numerical columns
    df_hourly[numerical_cols] = df_hourly[numerical_cols].interpolate(method='time')

    # Fill categorical columns with previous value or next value if previous is missing
    categorical_cols = ["hourly_present_weather_type", "hourly_sky_conditions"]
    df_hourly[categorical_cols] = df_hourly[categorical_cols].fillna(method='ffill').fillna(method='bfill')

    # Reset index to bring hourly_time back as a column
    df_hourly.reset_index(inplace=True)

    # Convert hourly_present_weather_type to descriptive terms
    df_hourly["hourly_present_weather_type"] = df_hourly["hourly_present_weather_type"].apply(interpret_weather_type)

    # Convert hourly_sky_conditions to descriptive terms
    df_hourly["hourly_sky_conditions"] = df_hourly["hourly_sky_conditions"].apply(interpret_sky_conditions)

    return df_hourly


##### Daily

In [ ]:
def clean_month_weather_data_daily(csv_file: str) -> pd.DataFrame:
    """
    Cleans the weather dataset for daily data by:
    - Extracting specific columns.
    - Interpreting the DailyWeather column using AU codes.
    - Converting Sunrise and Sunset columns to time format.
    - Replacing None in Sunrise and Sunset columns with the previous day's value.
    - Removing rows with NaN values.
    - Normalizing column names by adding underscores between words.
    - Replacing 'dry_bulb_temperature' with 'temperature' in column names.

    Parameters:
        csv_file (str): Path to the CSV file containing the weather data.

    Returns:
        pd.DataFrame: A cleaned DataFrame with selected and transformed columns.
    """
    # AU code mapping for DailyWeather
    au_code_mapping = {
        "DZ": "Drizzle",
        "RA": "Rain",
        "SN": "Snow",
        "SG": "Snow Grains",
        "IC": "Ice Crystals",
        "PL": "Ice Pellets",
        "GR": "Hail",
        "GS": "Small Hail",
        "UP": "Unknown Precipitation",
        "BR": "Mist",
        "FG": "Fog",
        "FU": "Smoke",
        "VA": "Volcanic Ash",
        "DU": "Dust",
        "SA": "Sand",
        "HZ": "Haze",
        "PY": "Spray",
        "PO": "Sand Whirls",
        "SQ": "Squalls",
        "FC": "Funnel Cloud",
        "SS": "Sandstorm",
        "DS": "Duststorm"
    }

    # Function to interpret DailyWeather using AU codes
    def interpret_daily_weather(weather_string):
        if pd.isnull(weather_string):
            return "Unknown"
        matches = re.findall(r"([A-Z]{2})[:\d]*", weather_string)
        descriptions = [au_code_mapping.get(code, "Unknown") for code in matches]
        return ", ".join(set(descriptions)) if descriptions else "Unknown"

    # Function to convert time in integer format (e.g., 720 -> 7:20 am)
    def convert_to_time_format(time_int):
        if pd.isnull(time_int):
            return None
        try:
            time_str = f"{int(time_int):04d}"  # Ensure it's always 4 digits (e.g., 720 -> "0720")
            return datetime.strptime(time_str, "%H%M").time()
        except ValueError:
            return None

    # Load the dataset
    df = pd.read_csv(csv_file, low_memory=False)

    # Select only the required columns
    columns_to_extract = [
        "DATE", "Sunrise", "Sunset", "DailyAverageDryBulbTemperature", 
        "DailyAverageWindSpeed", "DailyMaximumDryBulbTemperature", 
        "DailyMinimumDryBulbTemperature", "DailyWeather", "DailySnowDepth"
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]

    # Convert DATE to datetime
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["DATE"])  # Drop rows with invalid dates
    df = df.sort_values("DATE")  # Sort by date

    # Convert Sunrise and Sunset columns to time format
    if "Sunrise" in df.columns:
        df["Sunrise"] = df["Sunrise"].apply(convert_to_time_format)
    if "Sunset" in df.columns:
        df["Sunset"] = df["Sunset"].apply(convert_to_time_format)

    # Replace None in Sunrise and Sunset columns with the previous day's value
    df["Sunrise"] = df["Sunrise"].fillna(method='ffill')
    df["Sunset"] = df["Sunset"].fillna(method='ffill')

    # Interpret DailyWeather column
    if "DailyWeather" in df.columns:
        df["DailyWeather"] = df["DailyWeather"].apply(interpret_daily_weather)

    # Convert DailySnowDepth column to numeric and handle 'T' values
    if "DailySnowDepth" in df.columns:
        df["DailySnowDepth"] = df["DailySnowDepth"].replace("T", 0.01).astype(float)

    # Remove rows with NaN values
    df = df.dropna()

    # Rename columns to use underscores between words
    df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df.columns]

    # Replace 'dry_bulb_temperature' with 'temperature' in column names
    df.columns = [col.replace("dry_bulb_temperature", "temperature") for col in df.columns]

    df = df.rename(columns={"d_a_t_e": "date"})
    return df


In [ ]:
WEATHER_CSV_DIR = "/Users/victorzhang/Desktop/Columbia S1/IEOR E4501 Tools for Analytics/Final Project"
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather 
(
    hourly_time DATETIME PRIMARY KEY,
    hourly_temperature FLOAT,
    hourly_present_weather_type STRING,
    hourly_sky_conditions STRING,
    hourly_visibility FLOAT,
    hourly_precipitation FLOAT
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daliy_weather 
(
    date DATETIME PRIMARY KEY,
    sunrise TIME,
    sunset TIME,
    daily_average_temperature FLOAT,
    daily_average_wind_speed FLOAT,
    daily_maximum_temperature FLOAT,
    daily_minimum_temperature FLOAT,
    daily_weather STRING,
    daily_snow_depth FLOAT
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips 
(
    id INTEGER PRIMARY KEY,
    pickup_time DATETIME,
    dropoff_time DATETIME,
    passenger_count INTEGER,
    trip_distance FLOAT,
    fare_amount FLOAT,
    miscellaneous_extra_charges FLOAT,
    mta_tax FLOAT,
    tip_amount FLOAT,
    tolls_amount FLOAT,
    improvement_surcharge FLOAT,
    congestion_surcharge FLOAT,
    airport_fee FLOAT,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips 
(
    id INTEGER PRIMARY KEY,
    pickup_time DATETIME,
    dropoff_time DATETIME,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    trip_distance FLOAT,
    base_passenger_fare FLOAT,
    tolls FLOAT,
    sales_tax FLOAT,
    congestion_surcharge FLOAT,
    airport_fee FLOAT,
    tips FLOAT,
    driver_pay FLOAT,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT
)
"""


In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# Create tables using the updated schemas
with engine.connect() as connection:
    connection.execute(text(HOURLY_WEATHER_SCHEMA))
    connection.execute(text(DAILY_WEATHER_SCHEMA))
    connection.execute(text(TAXI_TRIPS_SCHEMA))
    connection.execute(text(UBER_TRIPS_SCHEMA))


### Add Data to Database

In [ ]:
def write_dataframes_to_table(
    table_to_df_dict: Dict[str, pd.DataFrame], 
    engine: Engine, 
    if_exists: str = "replace"
) -> None:
    """
    Write multiple DataFrames to a database as tables.

    Args:
        table_to_df_dict (Dict[str, pd.DataFrame]): A dictionary where keys are table names
                                                    and values are DataFrames to be written.
        engine (Engine): SQLAlchemy database engine.
        if_exists (str): Behavior if the table already exists. Options are 'fail', 'replace', or 'append'.
                         Default is 'replace'.

    Returns:
        None
    """
    for table_name, df in table_to_df_dict.items():
        try:
            df.to_sql(table_name, con=engine, if_exists=if_exists, index=False)
        except Exception as e:
            print(f"Error writing table '{table_name}': {e}")


In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

In [ ]:
def write_query_to_file(query: str, filename: str) -> None:
    """
    Writes a SQL query to a specified file.

    Parameters:
        query (str): The SQL query string to write to the file.
        filename (str): The name of the file where the query will be saved.

    Returns:
        None
    """
    try:
        with open(filename, "w") as f:
            f.write(query)
        print(f"Query successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query 1

In [ ]:
QUERY_1_FILENAME = "query1.sql"

QUERY_1 = """
SELECT 
    STRFTIME('%H', pickup_time) AS hour_of_day,
    COUNT(*) AS ride_count
FROM taxi_trips
WHERE pickup_time BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY hour_of_day
ORDER BY ride_count DESC;
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2

In [ ]:
QUERY_2_FILENAME = "query2.sql"

QUERY_2 = """
SELECT 
    strftime('%w', pickup_time) AS day_of_week, 
    COUNT(*) AS ride_count
FROM uber_trips
WHERE pickup_time BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY day_of_week
ORDER BY ride_count DESC; 
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_2)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_2, con=engine)

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3

In [ ]:
QUERY_3_FILENAME = "query3.sql"

QUERY_3 = """
WITH filtered_trips AS (
    SELECT 
        trip_distance
    FROM 
        uber_trips
    WHERE 
        DATE(pickup_time) BETWEEN '2024-01-01' AND '2024-01-31'
),
percentile_calculation AS (
    SELECT 
        trip_distance,
        ROW_NUMBER() OVER (ORDER BY trip_distance) AS row_num,
        COUNT(*) OVER () AS total_rows
    FROM 
        filtered_trips
)
SELECT 
    trip_distance
FROM 
    percentile_calculation
WHERE 
    row_num = CAST(total_rows * 0.95 AS INTEGER);
"""

In [ ]:
with engine.connect() as con:
    results = con.execute(db.text(QUERY_3)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_3, con=engine)

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4

In [ ]:
QUERY_4_FILENAME = "query4.sql"

QUERY_4 = """
WITH ride_counts AS (
    SELECT
        DATE(pickup_time) AS ride_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_distance
    FROM
        uber_trips
    WHERE
        pickup_time BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY
        DATE(pickup_time)
    UNION ALL
    SELECT
        DATE(pickup_time) AS ride_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_distance
    FROM
        taxi_trips
    WHERE
        pickup_time BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY
        DATE(pickup_time)
),
combined_counts AS (
    SELECT
        ride_date,
        SUM(total_rides) AS total_rides,
        AVG(avg_distance) AS avg_distance
    FROM
        ride_counts
    GROUP BY
        ride_date
),
top_10_days AS (
    SELECT
        ride_date,
        total_rides,
        avg_distance
    FROM
        combined_counts
    ORDER BY
        total_rides DESC
    LIMIT 10
)
SELECT
    t.ride_date,
    t.total_rides,
    t.avg_distance,
    AVG(w.hourly_precipitation) AS avg_precipitation,
    AVG(w.hourly_wind_speed) AS avg_wind_speed
FROM
    top_10_days t
JOIN
    hourly_weather w
ON
    DATE(w.hourly_time) = t.ride_date
GROUP BY
    t.ride_date, t.total_rides, t.avg_distance
ORDER BY
    t.total_rides DESC;
"""

In [ ]:
with engine.connect() as con:
    results = con.execute(db.text(QUERY_4)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_4, con=engine)

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5

In [ ]:
QUERY_5_FILENAME = "query5.sql"

QUERY_5 = """
WITH snow_days AS (
    SELECT
        DATE(date) AS snow_date,
        SUM(daily_snow_depth) AS total_snowfall
    FROM
        daily_weather
    WHERE
        date BETWEEN '2020-01-01 00:00:00' AND '2024-08-31 23:59:59'
        AND daily_snow_depth > 0
    GROUP BY
        DATE(date)
),
uber_rides AS (
    SELECT
        DATE(pickup_time) AS ride_date,
        COUNT(*) AS total_uber_rides
    FROM
        uber_trips
    WHERE
        pickup_time BETWEEN '2020-01-01 00:00:00' AND '2024-08-31 23:59:59'
    GROUP BY
        DATE(pickup_time)
),
taxi_rides AS (
    SELECT
        DATE(pickup_time) AS ride_date,
        COUNT(*) AS total_taxi_rides
    FROM
        taxi_trips
    WHERE
        pickup_time BETWEEN '2020-01-01 00:00:00' AND '2024-08-31 23:59:59'
    GROUP BY
        DATE(pickup_time)
),
total_rides AS (
    SELECT
        COALESCE(u.ride_date, t.ride_date) AS ride_date,
        COALESCE(u.total_uber_rides, 0) + COALESCE(t.total_taxi_rides, 0) AS total_rides
    FROM
        uber_rides u
    FULL OUTER JOIN
        taxi_rides t
    ON
        u.ride_date = t.ride_date
)
SELECT
    s.snow_date,
    s.total_snowfall,
    COALESCE(r.total_rides, 0) AS total_rides
FROM
    snow_days s
LEFT JOIN
    total_rides r
ON
    s.snow_date = r.ride_date
ORDER BY
    s.total_snowfall DESC
LIMIT 10;
"""

In [ ]:
with engine.connect() as con:
    results = con.execute(db.text(QUERY_5)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_5, con=engine)

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6

In [ ]:
QUERY_6_FILENAME = "query6.sql"

QUERY_6 = """
WITH RECURSIVE dates(x) AS (
    SELECT '2023-09-25 00:00:00' -- Start date and time
    UNION ALL
    SELECT datetime(x, '+1 hour')
    FROM dates
    WHERE x < '2023-10-03 23:00:00' -- End date and time
),
uber_hourly_rides AS (
    SELECT 
        strftime('%Y-%m-%d %H:00:00', pickup_time) AS hourly_time,
        COUNT(*) AS uber_rides
    FROM 
        uber_trips
    WHERE 
        pickup_time >= '2023-09-25 00:00:00' AND pickup_time <= '2023-10-03 23:59:59'
    GROUP BY 
        hourly_time
),
taxi_hourly_rides AS (
    SELECT 
        strftime('%Y-%m-%d %H:00:00', pickup_time) AS hourly_time,
        COUNT(*) AS taxi_rides
    FROM 
        taxi_trips
    WHERE 
        pickup_time >= '2023-09-25 00:00:00' AND pickup_time <= '2023-10-03 23:59:59'
    GROUP BY 
        hourly_time
),
hourly_rides AS (
    SELECT 
        COALESCE(u.hourly_time, t.hourly_time) AS hourly_time,
        COALESCE(u.uber_rides, 0) + COALESCE(t.taxi_rides, 0) AS num_rides
    FROM 
        uber_hourly_rides u
    FULL OUTER JOIN 
        taxi_hourly_rides t
    ON 
        u.hourly_time = t.hourly_time
),
hourly_weather_data AS (
    SELECT 
        strftime('%Y-%m-%d %H:00:00', hourly_time) AS hourly_time,
        SUM(hourly_precipitation) AS total_precipitation,
        AVG(hourly_wind_speed) AS avg_wind_speed
    FROM 
        hourly_weather
    WHERE 
        hourly_time BETWEEN '2023-09-25' AND '2023-10-03'
    GROUP BY 
        hourly_time
),
combined_data AS (
    SELECT 
        d.x AS hourly_time,
        COALESCE(r.num_rides, 0) AS num_rides,
        COALESCE(w.total_precipitation, 0) AS total_precipitation,
        COALESCE(w.avg_wind_speed, 0) AS avg_wind_speed
    FROM 
        dates d
    LEFT JOIN 
        hourly_rides r
    ON 
        d.x = r.hourly_time
    LEFT JOIN 
        hourly_weather_data w
    ON 
        d.x = w.hourly_time
)
SELECT 
    hourly_time,
    num_rides,
    total_precipitation,
    avg_wind_speed
FROM 
    combined_data
ORDER BY 
    hourly_time ASC;

"""


In [ ]:
with engine.connect() as con:
    results = con.execute(db.text(QUERY_6)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_6, con=engine)

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# Get the data for the first visualization
def get_data_for_visual_1():
    return pd.read_sql(QUERY_1, con=engine)

In [ ]:
def plot_most_popular_hour_for_taxis(dataframe: pd.DataFrame) -> None:
    """
    Create a bar chart to show the most popular hour to take a taxi, sorted from highest to lowest.

    Args:
        dataframe (pd.DataFrame): Contains columns 'hour_of_day' and 'ride_count'.

    Returns:
        None
    """
    # Sort the DataFrame by 'ride_count' in descending order
    sorted_data = dataframe.sort_values(by="ride_count", ascending=False)

    # Extract data from the sorted DataFrame
    hours = sorted_data["hour_of_day"].astype(str)  # Convert hours to strings for categorical x-axis
    ride_counts = sorted_data["ride_count"]

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(20, 10))

    # Create the bar plot
    ax.bar(hours, ride_counts, color="skyblue")

    # Add labels, title, and ticks
    ax.set_title("Most Popular Hour to Take a Taxi (2020-2024)", fontsize=16)
    ax.set_xlabel("Hour of the Day (24-hour format, sorted)", fontsize=14)
    ax.set_ylabel("Number of Rides", fontsize=14)
    ax.grid(axis="y", linestyle="--", alpha=0.7)  # Add grid lines for better readability

    # Display the plot
    plt.xticks(fontsize=12, rotation=45)  # Rotate x-axis labels for better readability
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()
    

In [ ]:
v_1 = get_data_for_visual_1()
plot_most_popular_hour_for_taxis(v_1)

### Visualization 2

In [ ]:
# Prepare the data for the second visualization
def get_combined_monthly_distance_data(
    uber_data: pd.DataFrame, 
    taxi_data: pd.DataFrame
) -> pd.DataFrame:
    """
    Combine Uber and Taxi trip data, filter by date range, and calculate the total distance traveled per month.

    Args:
        uber_data (pd.DataFrame): DataFrame containing Uber trip data with 'pickup_time' and 'trip_distance' columns.
        taxi_data (pd.DataFrame): DataFrame containing Taxi trip data with 'pickup_time' and 'trip_distance' columns.

    Returns:
        pd.DataFrame: A DataFrame with columns 'month' and 'total_distance' representing 
                      the total distance traveled per month for Uber and Taxi combined.
    """
    start_date = "2020-01-01"
    end_date = "2024-08-31"
    
    # Combine both datasets into one dataframe
    combined_data = pd.concat(
        [
            uber_data[["pickup_time", "trip_distance"]].assign(type="Uber"),
            taxi_data[["pickup_time", "trip_distance"]].assign(type="Taxi")
        ]
    )
    
    # Filter data by date range
    combined_data = combined_data[
        (combined_data["pickup_time"] >= start_date) & 
        (combined_data["pickup_time"] <= end_date)
    ]
    
    # Extract month
    combined_data["month"] = combined_data["pickup_time"].dt.month

    # Group by month and calculate total distance
    grouped_data = combined_data.groupby("month")["trip_distance"].sum().reset_index()
    grouped_data.rename(columns={"trip_distance": "total_distance"}, inplace=True)

    return grouped_data



In [ ]:
# Plot the total distance traveled per month for Uber and Taxi combined
def plot_monthly_total_distance(dataframe: pd.DataFrame) -> None:
    """
    Plot the total distance traveled per month for Uber and Taxi combined.

    Args:
        dataframe (pd.DataFrame): A DataFrame containing:
            - 'month': Month of the year (1-12).
            - 'total_distance': Total distance traveled in that month.

    Returns:
        None
    """
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=dataframe,
        x="month",
        y="total_distance",
        marker="o",
        errorbar=None
    )
    
    plt.title("Total Distance Traveled per Month (2020-2024)", fontsize=16)
    plt.xlabel("Month", fontsize=12)
    plt.ylabel("Total Distance (miles)", fontsize=12)
    plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
    plt.tight_layout()
    plt.show()


In [ ]:
v2_combined_data = get_combined_monthly_distance_data(uber_data, taxi_data)
plot_monthly_total_distance(v2_combined_data)


### Visualization 3

In [ ]:
# Prepare the data for the third visualization
airport_bboxes = {
    "LGA": {"lat_min": 40.766, "lat_max": 40.793, "lon_min": -73.896, "lon_max": -73.863},
    "JFK": {"lat_min": 40.641, "lat_max": 40.665, "lon_min": -73.79, "lon_max": -73.75},
    "EWR": {"lat_min": 40.683, "lat_max": 40.7, "lon_min": -74.2, "lon_max": -74.15},
}

def filter_rides_to_airport(
    df: pd.DataFrame, airport_name: str, bbox: Dict[str, float]
) -> pd.DataFrame:
    """
    Filter rides to a specific airport based on bounding box coordinates.

    Args:
        df (pd.DataFrame): DataFrame containing ride data with 'dropoff_latitude' and 'dropoff_longitude' columns.
        airport_name (str): Name of the airport (e.g., "LGA", "JFK", "EWR").
        bbox (Dict[str, float]): Bounding box dictionary with lat_min, lat_max, lon_min, and lon_max.

    Returns:
        pd.DataFrame: Filtered DataFrame of rides to the specified airport, with an 'airport' column added.
    """
    return df[
        (df["dropoff_latitude"] >= bbox["lat_min"]) &
        (df["dropoff_latitude"] <= bbox["lat_max"]) &
        (df["dropoff_longitude"] >= bbox["lon_min"]) &
        (df["dropoff_longitude"] <= bbox["lon_max"])
    ].assign(airport=airport_name)

def prepare_weekly_airport_ride_data(
    uber_data: pd.DataFrame, taxi_data: pd.DataFrame
) -> pd.DataFrame:
    """
    Prepare data for visualizing the number of rides to each airport per day of the week.

    Args:
        uber_data (pd.DataFrame): Uber ride data with 'dropoff_time', 'dropoff_latitude', and 'dropoff_longitude' columns.
        taxi_data (pd.DataFrame): Taxi ride data with 'dropoff_time', 'dropoff_latitude', and 'dropoff_longitude' columns.

    Returns:
        pd.DataFrame: A DataFrame grouped by 'airport' and 'day_of_week', with the number of rides as 'num_rides'.
    """
    filtered_data = []
    for airport, bbox in airport_bboxes.items():
        filtered_data.append(filter_rides_to_airport(uber_data, airport, bbox))
        filtered_data.append(filter_rides_to_airport(taxi_data, airport, bbox))
    

    airport_data = pd.concat(filtered_data)
    
    airport_data["day_of_week"] = airport_data["dropoff_time"].dt.day_name()
    
    grouped_data = airport_data.groupby(["airport", "day_of_week"]).size().reset_index(name="num_rides")
    
    day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    grouped_data["day_of_week"] = pd.Categorical(grouped_data["day_of_week"], categories=day_order, ordered=True)
    
    return grouped_data


In [ ]:
# Plot the number of rides to NYC airports by day of the week
def plot_rides_to_airports_by_weekday(dataframe: pd.DataFrame) -> None:
    """
    Plot the number of rides to NYC airports by day of the week.

    Args:
        dataframe (pd.DataFrame): A DataFrame containing:
            - 'day_of_week': The day of the week (e.g., "Monday").
            - 'num_rides': The number of rides to the airport.
            - 'airport': The name of the airport (e.g., "JFK", "LGA", "EWR").

    Returns:
        None
    """
    plt.figure(figsize=(16, 8))
    sns.barplot(
        data=dataframe,
        x="day_of_week",
        y="num_rides",
        hue="airport",
        palette="Set2"
    )
    plt.title("Most Popular Day of the Week for Rides to NYC Airports", fontsize=16)
    plt.xlabel("Day of Week", fontsize=12)
    plt.ylabel("Number of Rides", fontsize=12)
    plt.legend(title="Airport")
    plt.tight_layout()
    plt.show()

In [ ]:
v_3 = prepare_weekly_airport_ride_data(uber_data, taxi_data)
plot_rides_to_airports_by_weekday(v_3)


### Visualization 4

In [ ]:
def prepare_monthly_earnings_data(taxi_data: pd.DataFrame, uber_data: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare data for monthly earnings visualization from taxi and Uber datasets.

    Args:
        taxi_data (pd.DataFrame): Taxi trip data with 'pickup_time' and fare-related columns.
        uber_data (pd.DataFrame): Uber trip data with 'pickup_time' and fare-related columns.

    Returns:
        pd.DataFrame: A combined DataFrame with monthly earnings breakdown for Taxi and Uber rides.
    """
    # Extract month for grouping
    taxi_data["month"] = taxi_data["pickup_time"].dt.to_period("M")
    uber_data["month"] = uber_data["pickup_time"].dt.to_period("M")

    # Process Taxi Data
    taxi_grouped = taxi_data.groupby("month").agg(
        total_fare=("fare_amount", "sum"),
        base_fare=("fare_amount", "sum"),
        surcharges=("mta_tax", "sum"),
        tolls=("tolls_amount", "sum"),
        taxes=("tip_amount", "sum")
    ).reset_index()
    taxi_grouped["ride_type"] = "Taxi"

    # Process Uber Data
    uber_grouped = uber_data.groupby("month").agg(
        total_fare=("base_passenger_fare", "sum"),
        base_fare=("base_passenger_fare", "sum"),
        surcharges=("congestion_surcharge", "sum"),
        tolls=("tolls", "sum"),
        taxes=("sales_tax", "sum")
    ).reset_index()
    uber_grouped["ride_type"] = "Uber"

    # Combine Taxi and Uber Data
    combined_data = pd.concat([taxi_grouped, uber_grouped], ignore_index=True)

    # Ensure month is in datetime format for sorting
    combined_data["month"] = combined_data["month"].dt.to_timestamp()

    return combined_data



In [ ]:
def plot_monthly_earnings(dataframe: pd.DataFrame) -> None:
    """
    Plot the monthly earnings breakdown for taxis and Ubers.

    Args:
        dataframe (pd.DataFrame): A DataFrame containing:
            - 'month': The month of the earnings data (datetime format).
            - 'total_fare': Total earnings for the month.
            - 'ride_type': Type of ride ('Taxi' or 'Uber').

    Returns:
        None
    """
    plt.figure(figsize=(14, 8))
    sns.lineplot(
        data=dataframe,
        x="month",
        y="total_fare",
        hue="ride_type",
        marker="o"
    )
    plt.title("Monthly Total Earnings by Ride Type (2020-2024)", fontsize=16)
    plt.xlabel("Month", fontsize=12)
    plt.ylabel("Total Earnings ($)", fontsize=12)
    plt.legend(title="Ride Type")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()



In [ ]:
combined_data = prepare_monthly_earnings_data(taxi_data, uber_data)
plot_monthly_earnings(combined_data)


### Visualization 5

In [ ]:
# Prepare the data for the fifth visualization
def plot_tips_vs_factors_hourly(
    uber_data: pd.DataFrame, 
    taxi_data: pd.DataFrame, 
    hourly_weather_data: pd.DataFrame
) -> None:
    """
    Visualize the relationship between tips, distance, and precipitation for Uber and Yellow Taxi rides.

    Args:
        uber_data (pd.DataFrame): Uber trip data containing 'pickup_time', 'trip_distance', and 'tips'.
        taxi_data (pd.DataFrame): Taxi trip data containing 'pickup_time', 'trip_distance', and 'tips'.
        hourly_weather_data (pd.DataFrame): Weather data containing 'hourly_time' and 'hourly_precipitation'.

    Returns:
        None
    """
    if "tips" not in taxi_data.columns:
        taxi_data = taxi_data.rename(columns={"tip_amount": "tips"})
    
    start_date = "2022-01-01"
    end_date = "2023-12-31"
    
    uber_filtered = uber_data[(uber_data["pickup_time"] >= start_date) & (uber_data["pickup_time"] <= end_date)]
    taxi_filtered = taxi_data[(taxi_data["pickup_time"] >= start_date) & (taxi_data["pickup_time"] <= end_date)]
    
    hourly_weather_data["hourly_time"] = pd.to_datetime(hourly_weather_data["hourly_time"])
    taxi_filtered["hourly_time"] = taxi_filtered["pickup_time"].dt.floor("H")
    uber_filtered["hourly_time"] = uber_filtered["pickup_time"].dt.floor("H")
    
    taxi_filtered = pd.merge(
        taxi_filtered,
        hourly_weather_data,
        how="left",
        left_on="hourly_time",
        right_on="hourly_time"
    )
    uber_filtered = pd.merge(
        uber_filtered,
        hourly_weather_data,
        how="left",
        left_on="hourly_time",
        right_on="hourly_time"
    )
    
    # Remove unusual tips (e.g., tips over $100)
    taxi_filtered = taxi_filtered[taxi_filtered["tips"] < 100]
    uber_filtered = uber_filtered[uber_filtered["tips"] < 100]
    
    
    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle("Effect of Distance and Precipitation on Tips", fontsize=16)

    # 1：Yellow Taxi - Tips vs Distance
    sns.scatterplot(
        data=taxi_filtered,
        x="trip_distance",
        y="tips",
        ax=axes[0, 0],
        color="blue",
        alpha=0.6
    )
    axes[0, 0].set_title("Yellow Taxi: Tips vs Distance")
    axes[0, 0].set_xlabel("Trip Distance (miles)")
    axes[0, 0].set_ylabel("Tips (USD)")

    # 2：Uber - Tips vs Distance
    sns.scatterplot(
        data=uber_filtered,
        x="trip_distance",
        y="tips",
        ax=axes[0, 1],
        color="green",
        alpha=0.6
    )
    axes[0, 1].set_title("Uber: Tips vs Distance")
    axes[0, 1].set_xlabel("Trip Distance (miles)")
    axes[0, 1].set_ylabel("Tips (USD)")

    # 3：Yellow Taxi - Tips vs Precipitation
    sns.scatterplot(
        data=taxi_filtered,
        x="hourly_precipitation",
        y="tips",
        ax=axes[1, 0],
        color="purple",
        alpha=0.6
    )
    axes[1, 0].set_title("Yellow Taxi: Tips vs Precipitation")
    axes[1, 0].set_xlabel("Precipitation (inches)")
    axes[1, 0].set_ylabel("Tips (USD)")

    # 4：Uber - Tips vs Precipitation
    sns.scatterplot(
        data=uber_filtered,
        x="hourly_precipitation",
        y="tips",
        ax=axes[1, 1],
        color="orange",
        alpha=0.6
    )
    axes[1, 1].set_title("Uber: Tips vs Precipitation")
    axes[1, 1].set_xlabel("Precipitation (inches)")
    axes[1, 1].set_ylabel("Tips (USD)")

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

plot_tips_vs_factors_hourly(uber_data, taxi_data, hourly_weather_data)
    
    


### Visualization 6

In [ ]:
# Convert pickup_time to datetime in both datasets
uber_data['pickup_time'] = pd.to_datetime(uber_data['pickup_time'])
taxi_data['pickup_time'] = pd.to_datetime(taxi_data['pickup_time'])

# Filter for trips in 2020
uber_2020 = uber_data[uber_data['pickup_time'].dt.year == 2020]
taxi_2020 = taxi_data[taxi_data['pickup_time'].dt.year == 2020]

# Add a column to represent trip counts (1 per trip)
uber_2020['trips'] = 1
taxi_2020['trips'] = 1

# Combine Uber and Taxi data with relevant columns
combined_data = pd.concat([
    uber_2020[['pickup_latitude', 'pickup_longitude', 'trips']],
    taxi_2020[['pickup_latitude', 'pickup_longitude', 'trips']]
], ignore_index=True)

# Drop rows with NaN values in latitude or longitude
combined_data = combined_data.dropna(subset=['pickup_latitude', 'pickup_longitude'])

# Aggregate trip counts by unique pickup coordinates
heatmap_data = combined_data.groupby(['pickup_latitude', 'pickup_longitude'], as_index=False)['trips'].sum()

# Handle NaNs: Ensure no NaN values in latitude or longitude
heatmap_data = heatmap_data.dropna(subset=['pickup_latitude', 'pickup_longitude'])

# Prepare heatmap data as a list of [latitude, longitude, weight]
heatmap_data_list = heatmap_data[['pickup_latitude', 'pickup_longitude', 'trips']].values.tolist()

# Calculate map center (mean latitude and longitude), ignoring NaNs
map_center = [
    heatmap_data['pickup_latitude'].mean(skipna=True), 
    heatmap_data['pickup_longitude'].mean(skipna=True)
]

# Ensure map center has valid coordinates
if pd.isna(map_center[0]) or pd.isna(map_center[1]):
    print("Map center has invalid coordinates due to insufficient data.")
else:
    # Create a Folium map centered at the calculated mean coordinates
    heatmap_map = folium.Map(location=map_center, zoom_start=12)

    # Add a heatmap layer to the map
    HeatMap(heatmap_data_list, radius=10).add_to(heatmap_map)

display(heatmap_map)


### Extra credit 1:

In [ ]:
def animate_monthly_distance(dataframe: pd.DataFrame) -> HTML:
    """
    Create an animated line and scatter plot to visualize average distance traveled per month.

    Args:
        dataframe (pd.DataFrame): A DataFrame containing:
            - 'month': Month of the year (1-12).
            - 'total_distance': Total distance traveled for each month.

    Returns:
        HTML: An animation object displaying the visualization.
    """
    # Extract months and total distances
    months = dataframe["month"]
    distances = dataframe["total_distance"]

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 6))
    line, = ax.plot([], [], marker="o", color="blue", label="Total Distance Traveled")
    scatter = ax.scatter([], [], color="blue", s=50)

    # Set up plot aesthetics
    ax.set_xlim(1, 12)
    ax.set_ylim(0, distances.max() + 1000)
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
    ax.set_title("Average Distance Traveled per Month (2020-2024)", fontsize=16)
    ax.set_xlabel("Month", fontsize=12)
    ax.set_ylabel("Average Distance (miles)", fontsize=12)
    ax.legend()

    # Update function for animation
    def update(frame):
        line.set_data(months[:frame], distances[:frame])
        scatter.set_offsets(np.column_stack((months[:frame], distances[:frame])))
        return line, scatter

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(months), blit=True)

    plt.tight_layout()
    plt.show()
    return HTML(ani.to_jshtml())

# Display
v2_combined_data = get_combined_monthly_distance_data(uber_data, taxi_data)
animate_monthly_distance(v2_combined_data)

##### Extra Credit 4: What is the total number of hired rides within one hour before sunrise and sunset between 2023-12-15 and 2023-12-25?

In [ ]:
QUERY_7_FILENAME = "query7.sql"

QUERY_7 = """
WITH daily_sunrise_sunset AS (
    SELECT
        DATE(date) AS ride_date,
        TIME(sunrise) AS sunrise_time,
        TIME(sunset) AS sunset_time
    FROM
        daily_weather
    WHERE
        date BETWEEN '2023-12-15' AND '2023-12-25'
),
sunrise_sunset_periods AS (
    SELECT
        ride_date,
        datetime(ride_date || ' ' || sunrise_time, '-1 hour') AS sunrise_start,
        datetime(ride_date || ' ' || sunrise_time, '+1 hour') AS sunrise_end,
        datetime(ride_date || ' ' || sunset_time, '-1 hour') AS sunset_start,
        datetime(ride_date || ' ' || sunset_time, '+1 hour') AS sunset_end
    FROM
        daily_sunrise_sunset
),
rides_sunrise_sunset AS (
    SELECT
        sp.ride_date,
        COUNT(CASE WHEN u.pickup_time BETWEEN sp.sunrise_start AND sp.sunrise_end THEN 1 END) +
        COUNT(CASE WHEN t.pickup_time BETWEEN sp.sunrise_start AND sp.sunrise_end THEN 1 END) AS total_rides_sunrise,
        COUNT(CASE WHEN u.pickup_time BETWEEN sp.sunset_start AND sp.sunset_end THEN 1 END) +
        COUNT(CASE WHEN t.pickup_time BETWEEN sp.sunset_start AND sp.sunset_end THEN 1 END) AS total_rides_sunset
    FROM
        sunrise_sunset_periods sp
    LEFT JOIN
        uber_trips u
    ON
        DATE(u.pickup_time) = sp.ride_date
    LEFT JOIN
        taxi_trips t
    ON
        DATE(t.pickup_time) = sp.ride_date
    WHERE
        (u.pickup_time BETWEEN sp.sunrise_start AND sp.sunrise_end
         OR u.pickup_time BETWEEN sp.sunset_start AND sp.sunset_end)
        OR
        (t.pickup_time BETWEEN sp.sunrise_start AND sp.sunrise_end
         OR t.pickup_time BETWEEN sp.sunset_start AND sp.sunset_end)
    GROUP BY
        sp.ride_date
)
SELECT
    ride_date,
    total_rides_sunrise,
    total_rides_sunset
FROM
    rides_sunrise_sunset
ORDER BY
    ride_date ASC;
"""

In [ ]:
with engine.connect() as con:
    results = con.execute(db.text(QUERY_7)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_7, con=engine)

In [ ]:
# Plotting the data
data = pd.read_sql(QUERY_7, con=engine)


# Convert ride_date to datetime for better plotting
data['ride_date'] = pd.to_datetime(data['ride_date'])

# Plot settings
fig, ax = plt.subplots(figsize=(12, 6))

# Bar width
bar_width = 0.4

# X-axis positions for bars
x = range(len(data))

# Plot Sunrise and Sunset rides as side-by-side bars
bars_sunrise = ax.bar(x, data['total_rides_sunrise'], width=bar_width, label='Sunrise Rides', color='skyblue', alpha=0.8)
bars_sunset = ax.bar([p + bar_width for p in x], data['total_rides_sunset'], width=bar_width, label='Sunset Rides', color='orange', alpha=0.8)

# X-axis labels
ax.set_xticks([p + bar_width / 2 for p in x])
ax.set_xticklabels(data['ride_date'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=10)

# Add labels on top of bars
for bar in bars_sunrise:
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 1,
        str(bar.get_height()),
        ha='center',
        va='bottom',
        fontsize=10
    )

for bar in bars_sunset:
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 1,
        str(bar.get_height()),
        ha='center',
        va='bottom',
        fontsize=10
    )

# Labels and title
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Total Rides', fontsize=12)
ax.set_title('Total Rides During Sunrise and Sunset', fontsize=14)
ax.legend(fontsize=10)

# Grid for better readability
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()